In [32]:
!pip install torch
!pip install fastai==1.0.61

In [33]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from typing import Dict
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [34]:
RENTTHERUNWAY_1_FILE_PATH = Path('./renttherunway_data_1.json')
rtr_df_1 = pd.read_json(RENTTHERUNWAY_1_FILE_PATH, lines=True)


In [35]:
rtr_df_1.shape

(99999, 15)

## Clean Text

In [36]:
rtr_df_1['review_text'] = rtr_df_1['review_text'].str.replace("[^a-zA-Z]", " ")
nltk.download('stopwords')
stop_words = stopwords.words('english')
tokenized_doc = rtr_df_1['review_text'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
detokenized_doc = [] 
for i in range(len(rtr_df_1)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

rtr_df_1['review_text'] = detokenized_doc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
rtr_df_1_trn, rtr_df_1_val = train_test_split(rtr_df_1, stratify = rtr_df_1['fit'], test_size = 0.4, random_state = 12)

In [38]:
rtr_df_1_trn.head()


,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
66521,fit,123373,36b,1963564,165lbs,8.0,work,I probably gotten size little form fitting bac...,athletic,I got this for work and received a lot of comp...,dress,"5' 4""",16,37.0,"April 22, 2016"
9782,fit,768673,30d,1589928,120lbs,10.0,date,Extra small usually fits well company although...,athletic,Colorful and Cute,sheath,"5' 4""",4,28.0,"December 1, 2016"
84578,small,559612,34d,255476,130lbs,6.0,wedding,The back dress cute I really wanted love SO SH...,hourglass,cute but just not for me,dress,"5' 3""",4,30.0,"November 10, 2016"
4020,small,105716,32d,1289088,140lbs,4.0,date,The dress TOO form fitting It fit top much tig...,hourglass,"Pretty dress, but unable to wear",sheath,"5' 3""",14,30.0,"February 19, 2016"
6670,fit,832196,36c,1166477,NaN,10.0,other,Vibrant beautiful I absolutely loved dress Run...,hourglass,RTR Stylist Review!,dress,"5' 1""",8,24.0,"March 23, 2017"


In [39]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df=rtr_df_1_trn, valid_df=rtr_df_1_val, path=".", text_cols=["review_text","review_summary"], label_cols=["fit"])

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = ".", train_df = rtr_df_1_trn, valid_df = rtr_df_1_val, vocab=data_lm.train_ds.vocab, bs=32, text_cols=["review_text","review_summary"], label_cols=["fit"])

## Language Model

In [49]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3,pretrained=True, metrics=[accuracy])

In [50]:
learn.fit_one_cycle(1, 1.45e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.280101,4.166524,0.251884,02:23


In [51]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.005758,3.987941,0.271796,03:11
1,3.878005,3.866979,0.284535,03:11
2,3.783081,3.795686,0.290858,03:11
3,3.650695,3.755513,0.294689,03:11
4,3.539890,3.747478,0.295437,03:11
5,3.415979,3.761856,0.295384,03:11
6,3.289856,3.787978,0.293524,03:12
7,3.167706,3.820886,0.291194,03:12
8,3.080592,3.847016,0.289454,03:12
9,3.035021,3.857825,0.289078,03:11


In [52]:
learn.save_encoder('finetuned')

## Classifier

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.7, metrics=[accuracy, Precision(), Recall()])
learn.load_encoder('finetuned')

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.7, metrics=[accuracy])
learn.load_encoder('finetuned')

In [55]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.675225,0.633839,0.758825,01:41


In [56]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.564067,0.475312,0.826875,01:51


In [57]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.507975,0.458914,0.834550,02:55


In [58]:
learn.unfreeze()
learn.fit_one_cycle(4, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.505037,0.460822,0.833375,03:18
1,0.468482,0.453184,0.836625,03:49
2,0.484756,0.448320,0.838000,03:17
3,0.459408,0.448477,0.837800,03:43
